In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name, when

In [ ]:
spark = SparkSession.builder \
                    .appName("Healthcare Claims Ingestion") \
                    .getOrCreate()

In [ ]:
BUCKET_NAME = "healthcare-bucket-0301"
CLAIMS_BUCKET_PATH = f"gs://{BUCKET_NAME}/landing/claims/*.csv"
BQ_TABLE = "learning-0301.bronze_dataset.claims"
TEMP_GCS_BUCKET = f"{BUCKET_NAME}/temp/"

In [ ]:
claims_df = spark.read.csv(CLAIMS_BUCKET_PATH, header=True)

In [ ]:
claims_df = (claims_df
                .withColumn("datasource", 
                              when(input_file_name().contains("hospital2"), "hosb")
                             .when(input_file_name().contains("hospital1"), "hosa").otherwise("None")))

In [ ]:
claims_df = claims_df.dropDuplicates()

In [ ]:
(claims_df.write
            .format("bigquery")
            .option("table", BQ_TABLE)
            .option("temporaryGcsBucket", TEMP_GCS_BUCKET)
            .mode("overwrite")
            .save())